In [1]:
import cv2
from google_images_download import google_images_download
import matplotlib.pyplot as plt
import imutils
import numpy as np
import glob
import turicreate as tc
import json

/home/kabirkhan/.local/share/virtualenvs/ArchRecognition-VDW7_LdE/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
def search_and_download_images(keywords, num_images=40):

    response = google_images_download.googleimagesdownload() 

    arguments = {
        "keywords": keywords,
        "limit": num_images,
        "print_urls": True,
        "prefix_keywords": 'azure',
        "suffix_keywords": 'icon',
        "format": 'png'
    }
    absolute_image_paths = response.download(arguments)

    with open('./image_urls', 'w+') as image_urls_file:
        for k, v in absolute_image_paths.items():
            image_urls_file.write(k)
            image_urls_file.write('=' * 80)
            image_urls_file.writelines(list(absolute_image_paths.values()).join('\n\r'))
            
    return absolute_image_paths

search_and_download_images('database,virtual machine,webapp,file storage')

In [ ]:
glob.glob('./downloads/**/*.png')

In [24]:
image = cv2.imread('../data/images/Whiteboard2arm-data-resized/20180723_021702377_iOS.jpg')
orig = image.copy()
image = imutils.resize(image, height=300)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 100, 50, 50)



cv2.imwrite('./proccessed.png', inverted_edged)

# cv2.imshow("Inverted Edge", inverted_edged)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True

In [22]:
# cv2.threshold(image, gray, 0, 255.0, cv2.THRESH_OTSU | cv2.THRESH_BINARY)

cv2.imwrite('./dilated.png', dilation)
# element = cv2.getStructuringElement(Imgproc.MORPH_RECT, 7, 7)
# cv2.dilate(grayImg, grayImg2, element);
# cv2.erode(grayImg2, grayImg, element);

True

In [30]:
from urllib.parse import quote_plus

quote_plus('Test 4')

'Test+4'

In [28]:
def create_edge_path_image(img_path, verbose=False):
    if verbose:
        print(img_path)
    image = cv2.imread(img_path)
    image = imutils.resize(image, height=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 75, 125)
    
    # Dilate to combine close edges into one line
    kernel = np.ones((5,5), np.uint8)
    dilated = cv2.dilate(edged, kernel, iterations=1)

    # Invert edge detection to get black lines on white background (whiteboard representative)
    inverted_edged = cv2.bitwise_not(dilated)
    
    
    edge_path = img_path[:-4] + '-processed.png'
    
    cv2.imwrite(edge_path, inverted_edged)
    
    
create_edge_path_image('../data/images/Whiteboard2arm-data-resized/20180723_024505078_iOS.jpg')

In [54]:
for png_path in glob.glob('./images/*.jpeg'):
    create_edge_path_image(png_path)
    
    
# [{'coordinates': {'height': 104, 'width': 110, 'x': 115, 'y': 216},
#   'label': 'ball'},

In [32]:
tc_object_detect_images = []
tc_object_detect_annotations = []

with open('./annotations/2/wb2arm_images_2.jsonl') as annotations_file:
    for line in annotations_file:
        annotations = json.loads(line)
        if annotations['answer'] == 'accept':
            boxes = annotations['spans']
            
#             print(annotations)
            object_detection_boxes_out = []
            for box in boxes:

                points = box['points']
                top_left, bottom_left, bottom_right, top_right = points
                print(top_left, top_right, bottom_right, bottom_left)
                width = float(top_right[0] - top_left[0])
                height = float(bottom_left[1] - top_left[1])
                x_center = float(top_left[0] + (width / 2))
                y_center = float(top_left[1] + (height / 2))

                object_detection_boxes_out.append({
                    'label': box['label'],
                    'coordinates': {
                        'height': height,
                        'width': width,
                        'x': x_center,
                        'y': y_center
                    }
                })
                
            print(annotations['meta']['file'])
            tc_object_detect_images.append(annotations['meta']['file'])
            tc_object_detect_annotations.append(object_detection_boxes_out)
            
            
annotations_sf = tc.SFrame({'filename': tc_object_detect_images, 'annotations': tc_object_detect_annotations})

[86.3, 102.1] [665, 102.1] [665, 339.8] [86.3, 339.8]
20180723_024521307_iOS.jpg
[251.5, 45.2] [562.9, 45.2] [562.9, 297.8] [251.5, 297.8]
20180723_024512276_iOS.jpg
[38, 47] [355, 47] [355, 377] [38, 377]
20180723_021708849_iOS.jpg
[92, 73] [281, 73] [281, 311] [92, 311]
20180723_021702377_iOS.jpg
[392.5, 42.1] [599.7, 42.1] [599.7, 264.1] [392.5, 264.1]
[170.4, 155.7] [320.9, 155.7] [320.9, 341.9] [170.4, 341.9]
[41, 240.9] [119.9, 240.9] [119.9, 328.3] [41, 328.3]
20180723_024439726_iOS.jpg
[127.3, 131.5] [279.9, 131.5] [279.9, 349.3] [127.3, 349.3]
[335.6, 42.1] [436.6, 42.1] [436.6, 158.9] [335.6, 158.9]
[377.7, 216.7] [580.8, 216.7] [580.8, 447.2] [377.7, 447.2]
20180723_022618958_iOS.jpg
[293.6, 87.3] [460.8, 87.3] [460.8, 318.8] [293.6, 318.8]
20180723_024539975_iOS.jpg
[69, 108] [148, 108] [148, 198] [69, 198]
[105, 236] [247, 236] [247, 385] [105, 385]
[303, 265] [383, 265] [383, 362] [303, 362]
20180723_021705068_iOS.jpg
[233.6, 143.1] [473.5, 143.1] [473.5, 404] [233.6, 404

In [24]:
for path in glob.glob('./images/Whiteboard2arm-data/*'):
    image = cv2.imread(path)
    try:
        
        image = imutils.resize(image, height=500)

        filename = path.split('/')[-1]
        path = '/'.join(path.split('/')[:-2]) + '/Whiteboard2arm-data-resized/'
        print(path + filename)
        cv2.imwrite(path + filename, image)
    except:
        pass

./images/Whiteboard2arm-data-resized/20180723_024521307_iOS.jpg
./images/Whiteboard2arm-data-resized/20180723_024512276_iOS.jpg
./images/Whiteboard2arm-data-resized/20180723_021708849_iOS.jpg
./images/Whiteboard2arm-data-resized/20180723_021702377_iOS.jpg
./images/Whiteboard2arm-data-resized/20180723_024439726_iOS.jpg
./images/Whiteboard2arm-data-resized/20180723_022618958_iOS.jpg
./images/Whiteboard2arm-data-resized/20180723_024539975_iOS.jpg
./images/Whiteboard2arm-data-resized/20180723_021705068_iOS.jpg
./images/Whiteboard2arm-data-resized/20180723_024531838_iOS.jpg
./images/Whiteboard2arm-data-resized/20180723_024501745_iOS.jpg
./images/Whiteboard2arm-data-resized/20180723_024450922_iOS.jpg
./images/Whiteboard2arm-data-resized/20180723_022635236_iOS.jpg
./images/Whiteboard2arm-data-resized/20180723_024433494_iOS.jpg
./images/Whiteboard2arm-data-resized/20180723_024507610_iOS.jpg
./images/Whiteboard2arm-data-resized/20180723_021711073_iOS.jpg
./images/Whiteboard2arm-data-resized/201

In [33]:
images = tc.image_analysis.load_images('./images/Whiteboard2arm-data-resized/', recursive=True)
images['filename'] = images['path'].apply(lambda s: s.split('/')[-1])

In [34]:
tc_object_detect_data = images.join(annotations_sf, on='filename')[['image', 'annotations']]
tc_object_detect_data

image,annotations
Height: 500 Width: 375,"[{'label':'SQL_DATABASE', ..."
Height: 500 Width: 375,"[{'label':'SQL_DATABASE', ..."
Height: 500 Width: 375,"[{'label':'SQL_DATABASE', ..."
Height: 500 Width: 375,"[{'label':'SQL_DATABASE', ..."
Height: 500 Width: 666,"[{'label':'SQL_DATABASE', ..."
Height: 500 Width: 666,"[{'label':'SQL_DATABASE', ..."
Height: 500 Width: 666,"[{'label':'SQL_DATABASE', ..."
Height: 500 Width: 666,"[{'label':'SQL_DATABASE', ..."
Height: 500 Width: 666,"[{'label':'SQL_DATABASE', ..."
Height: 500 Width: 666,"[{'label':'DOCUMENT_DATABASE', ..."


In [35]:
train_data, test_data = tc_object_detect_data.random_split(0.8)


In [36]:
train_data.explore()

Materializing SFrame...

Done.

In [ ]:
# Create a model
model = tc.object_detector.create(train_data, verbose=True)

Using 'image' as feature column
Using 'annotations' as annotations column
2018-07-22 23:52:58  Training    1/1000  Loss  8.086
2018-07-22 23:53:17  Training    3/1000  Loss  8.201
2018-07-22 23:53:37  Training    5/1000  Loss  8.220
2018-07-22 23:53:56  Training    7/1000  Loss  8.215
2018-07-22 23:54:16  Training    9/1000  Loss  8.246
2018-07-22 23:54:35  Training   11/1000  Loss  8.223
2018-07-22 23:54:54  Training   13/1000  Loss  8.050
2018-07-22 23:55:14  Training   15/1000  Loss  7.968
2018-07-22 23:55:34  Training   17/1000  Loss  7.790
2018-07-22 23:55:54  Training   19/1000  Loss  7.588
2018-07-22 23:56:04  Training   20/1000  Loss  7.471
2018-07-22 23:56:24  Training   22/1000  Loss  7.191
2018-07-22 23:56:44  Training   24/1000  Loss  6.907
2018-07-22 23:57:04  Training   26/1000  Loss  6.623
2018-07-22 23:57:14  Training   27/1000  Loss  6.449
2018-07-22 23:57:24  Training   28/1000  Loss  6.287
2018-07-22 23:57:34  Training   29/1000  Loss  6.162
2018-07-22 23:57:54  Trai

In [ ]:

# Save predictions to an SArray
predictions = model.predict(test_data)

# Evaluate the model and save the results into a dictionary
metrics = model.evaluate(test_data)